In [1]:
import numpy as np
import pandas as pd

In [2]:
train = pd.read_csv("training.csv")
test = pd.read_csv("testing.csv")

In [3]:
data = pd.concat([train, test])

#### Adding a quarter feature

In [4]:
data['quarter'] = data.Date.str.split('-',expand=True).drop([0,2],axis=1)

In [5]:
data_df = data.copy()

#### Dropping NaN columns

In [6]:
data_df.dropna(axis = 1, how = "all", inplace = True)

In [7]:
data_df.head()

,Date,Identifier,Dep_Var,Gr1Ind_Var1,Gr1Ind_Var2,Gr1Ind_Var3,Gr1Ind_Var4,Gr1Ind_Var5,Gr1Ind_Var6,Gr1Ind_Var7,...,Gr2Ind_Var4,Gr2Ind_Var5,Gr2Ind_Var6,Gr2Ind_Var7,Gr2Ind_Var8,Gr2Ind_Var9,Gr2Ind_Var10,Gr2Ind_Var11,Gr2Ind_Var12,quarter
0,2012-10-22,1855,16.0,91624.526353,0.981399,-0.032599,0.030106,0.010643,0.030998,0.703471,...,0.0,0.181818,0.0,0.0,0.0,0.0,0.0,0.909091,0.165289,10
1,2012-10-22,9079,13.0,472.315066,0.980664,0.058861,-0.010248,-0.012876,-0.010084,-0.124329,...,0.0,0.125000,0.0,0.0,0.0,0.0,0.0,0.750000,0.093750,10
2,2012-10-22,49725,16.0,1039.574650,0.852444,0.072414,0.011823,0.002200,0.004664,0.101501,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.600000,0.000000,10
3,2012-10-22,9569,9.0,4241.026023,1.094491,-0.015389,0.004078,0.002119,0.002838,0.079300,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.857143,0.000000,10
4,2012-10-22,6828,6.0,15404.194258,1.020933,0.009126,-0.003701,-0.003559,-0.017787,-0.335150,...,0.0,0.076923,0.0,0.0,0.0,0.0,0.0,0.615385,0.047337,10


#### Creating correlation matrix between columns and then dropping >0.95 correlated columns (keep only one)

In [8]:
# Create correlation matrix
corr_matrix = data_df.corr().abs()# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))# Find index of feature columns with correlation greater than 0.95
to_drop = [column for column in upper.columns if any(upper[column] > 0.95)]
print(f"Dropped Columns : {to_drop}")
data_df.drop(to_drop, axis=1, inplace = True)

Dropped Columns : ['Gr1Ind_Var8', 'Gr1Ind_Var12', 'Gr1Ind_Var18', 'Gr1Ind_Var31', 'Gr1Ind_Var48', 'Gr1Ind_Var49', 'Gr1Ind_Var50', 'Gr1Ind_Var84', 'Gr2Ind_Var4', 'Gr2Ind_Var7', 'Gr2Ind_Var8', 'Gr2Ind_Var12']


#### Fill empty cells by median grouped by date and then min-max transforming

In [9]:
data_df.drop(["Identifier", "Dep_Var"], axis = 1, inplace = True)

In [10]:
data_df = data_df.groupby("Date").transform(lambda x: x.fillna(x.median()))
data_df = pd.concat([data_df, data["Date"]], axis = 1)
data_df = data_df.groupby("Date").transform(lambda x: (x - x.min()) / (x.max() - x.min()))

In [11]:
data_df.head()

,Gr1Ind_Var1,Gr1Ind_Var2,Gr1Ind_Var3,Gr1Ind_Var4,Gr1Ind_Var5,Gr1Ind_Var6,Gr1Ind_Var7,Gr1Ind_Var9,Gr1Ind_Var10,Gr1Ind_Var11,...,Gr1Ind_Var102,Gr1Ind_Var103,Gr1Ind_Var104,Gr2Ind_Var1,Gr2Ind_Var3,Gr2Ind_Var5,Gr2Ind_Var6,Gr2Ind_Var9,Gr2Ind_Var10,Gr2Ind_Var11
0,0.138977,0.100411,0.296985,0.630448,0.166149,0.093769,0.098546,0.178036,0.669762,0.465988,...,0.702452,0.537637,0.695183,0.285714,0.0,0.290909,0.0,0.0,0.0,0.909091
1,0.000715,0.100284,0.348129,0.620601,0.165853,0.093434,0.098287,0.191915,0.671485,0.562379,...,0.712368,0.541449,0.695077,0.142857,0.0,0.200000,0.0,0.0,0.0,0.750000
2,0.001575,0.078100,0.355708,0.625987,0.166043,0.093554,0.098357,0.170096,0.670169,0.474765,...,0.749933,0.547404,0.695309,0.000000,0.0,0.000000,0.0,0.0,0.0,0.600000
3,0.006432,0.119977,0.306609,0.624097,0.166042,0.093539,0.098350,0.175814,0.670056,0.465680,...,0.632483,0.557796,0.692990,0.000000,0.0,0.000000,0.0,0.0,0.0,0.857143
4,0.023364,0.107251,0.320318,0.622199,0.165970,0.093371,0.098221,0.176851,0.669367,0.461988,...,0.706293,0.541449,0.694713,0.142857,0.0,0.123077,0.0,0.0,0.0,0.615385


In [12]:
data_df = np.log1p(data_df)

In [13]:
data_df = pd.concat([data["Date"], data_df], axis = 1).groupby("Date").transform(lambda x : (x - x.mean()) / x.std())

In [14]:
data_df = pd.concat([data[["Date", "Dep_Var", "Identifier", "quarter"]], data_df], axis = 1)

In [15]:
data_df = pd.concat([data_df, pd.get_dummies(data["quarter"])], axis = 1)

In [16]:
data_df.head()

,Date,Dep_Var,Identifier,quarter,Gr1Ind_Var1,Gr1Ind_Var2,Gr1Ind_Var3,Gr1Ind_Var4,Gr1Ind_Var5,Gr1Ind_Var6,...,Gr2Ind_Var3,Gr2Ind_Var5,Gr2Ind_Var6,Gr2Ind_Var9,Gr2Ind_Var10,Gr2Ind_Var11,01,04,07,10
0,2012-10-22,16.0,1855,10,3.364213,-0.118776,-0.518465,0.244902,-0.037887,-0.010729,...,-0.612084,0.626761,-0.163014,-0.190677,-0.588934,1.045256,0,0,0,1
1,2012-10-22,13.0,9079,10,-0.299249,-0.121315,0.491515,-0.106856,-0.058163,-0.035781,...,-0.612084,0.060773,-0.163014,-0.190677,-0.588934,-0.107230,0,0,0,1
2,2012-10-22,16.0,49725,10,-0.274920,-0.568746,0.637914,0.085801,-0.045165,-0.026787,...,-0.612084,-1.352328,-0.163014,-0.190677,-0.588934,-1.294163,0,0,0,1
3,2012-10-22,9.0,9569,10,-0.138004,0.268391,-0.325414,0.018267,-0.045235,-0.027901,...,-0.612084,-1.352328,-0.163014,-0.190677,-0.588934,0.679847,0,0,0,1
4,2012-10-22,6.0,6828,10,0.334293,0.017343,-0.052851,-0.049639,-0.050130,-0.040480,...,-0.612084,-0.452700,-0.163014,-0.190677,-0.588934,-1.167414,0,0,0,1


#### Splitting into train, validation, test

In [17]:
data_df["Date"].unique()

array(['2012-10-22', '2013-01-20', '2013-04-21', '2013-07-22',
       '2013-10-22', '2014-01-21', '2014-04-22', '2014-07-23',
       '2014-10-23', '2015-01-21', '2015-04-22', '2015-07-23',
       '2015-10-23', '2016-01-21', '2016-04-21', '2016-07-22'],
      dtype=object)

In [18]:
tr = ['2012-10-22', '2013-01-20', '2013-04-21', '2013-07-22',
       '2013-10-22', '2014-01-21', '2014-04-22', '2014-07-23']
vl = ['2014-10-23', '2015-01-21', '2015-04-22', '2015-07-23']
tt = ['2015-10-23', '2016-01-21', '2016-04-21', '2016-07-22']

In [19]:
train = data_df[data_df["Date"].isin(tr)]
val = data_df[data_df["Date"].isin(vl)]
test = data_df[data_df["Date"].isin(tt)]